<a href="https://colab.research.google.com/github/ykim71/georgia_analysis/blob/main/Georgia_all_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# need access to the folder(s) below:

#%cd /content/drive/MyDrive/Georgia/


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Locations Analysis


## Data load/prep for translation

### previous data

In [ ]:
# not needed anymore
#%cd /content/drive/MyDrive/Marquee data

import pandas as pd
import os
path = os.getcwd()
files = os.listdir(path)

files_xls = [f for f in files if f[-4:] == 'xlsx']
files_xls


['addl_2_2023-01-30.xlsx',
 'batch_1_2023-01-30.xlsx',
 'batch_2_2023-01-30.xlsx',
 'batch_3_2023-01-30.xlsx',
 'batch_4_2023-01-30.xlsx',
 'addl_1_2023-01-30.xlsx']

In [ ]:
df_list = []

for f in files_xls:
    data = pd.read_excel(f, dtype=object)
    df_list.append(data)

In [ ]:
df_frame = pd.concat(df_list, axis=0, ignore_index=True)


In [ ]:
df_frame_nna = df_frame.dropna(subset=['article_text'])

In [ ]:
df_frame_nna = df_frame_nna.sort_values(by='source')

In [ ]:
samples_count = df_frame_nna.groupby('source').count()

In [ ]:
samples_count

In [ ]:
min(df_frame_nna.date_published)

'2022-09-15'

In [ ]:
max(df_frame_nna.date_published)

'2022-10-15'

In [ ]:
len(df_frame)

215346

In [ ]:
len(df_frame_nna)

212997

In [ ]:
# remove line breaks
df_frame_nna['article_text'] = df_frame_nna['article_text'].astype(str)
df_frame_nna['article_text'] = df_frame_nna['article_text'].map(lambda x: x.replace('\n', ' '))

In [ ]:
%cd /content/drive/MyDrive/Georgia

df_frame_nna.to_pickle('Georgia_data_all.pkl')

/content/drive/MyDrive/Georgia


In [ ]:
import pandas as pd

df_frame_nna = pd.read_pickle('Georgia_data_all.pkl')

In [ ]:
import pandas as pd

df_frame_nna = pd.read_pickle('Georgia_data_all_ner_lem.pkl')

In [ ]:
count = df_frame_nna.groupby('source')['article_text'].count()

In [ ]:
count.to_excel('Georgia_source_list.xlsx')

### updated version

In [ ]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/Georgia (1)/Web Scraping /Updated Marquee data/delivery_2023-03-02.xlsx')

In [ ]:
df.columns

Index(['source', 'website', 'article_url', 'article_title', 'article_author',
       'date_published', 'article_text'],
      dtype='object')

In [ ]:
temp = df.groupby(['source']).count().reset_index()
temp[temp['source'].str.contains("Northside Neighbor|Paulding Neighbor|South Fulton Neighbor|Marietta Daily Journal")]
# Northside Neighbor, Paulding Neighbor, South Fulton Neighbor, Marietta Daily Journal

,source,website,article_url,article_title,article_author,date_published,article_text
120,Marietta Daily Journal,11695,11695,11695,9739,11695,11619
132,Northside Neighbor,4601,4601,4601,4096,4601,4599
138,Paulding Neighbor,4416,4416,4416,3979,4416,4414
154,South Fulton Neighbor,7354,7354,7354,6565,7354,7351


In [ ]:
df_nna = df.dropna(subset=['article_text'])

In [ ]:
print(len(df_nna))
print(len(df))

192206
214945


In [ ]:
df_nna_sample = df_nna.groupby('source').apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)
df_nna_sample.to_excel('df_nna_sample.xlsx')

In [ ]:
len(df_nna_sample)

2310

### translation

In [ ]:
%cd /content/drive/MyDrive/Georgia_YK

In [ ]:
df_sample_lang = pd.read_excel('df_nna_sample_wlang.xlsx')

In [ ]:
df_sample_lang = df_sample_lang[['source', 'LANGUAGE']].dropna(subset=['LANGUAGE']).drop_duplicates()

In [ ]:
df_sample_lang_article = pd.merge(df_sample_lang, df_nna, how="left", on="source")

In [ ]:
df_sample_lang_article.to_excel('df_sample_lang_article_translation.xlsx')

In [ ]:
#df_sample_lang_article[0:5000].to_excel('df_sample_lang_article_translation_file1.xlsx')
df_sample_lang_article[5000:10000].to_excel('df_sample_lang_article_translation_file2.xlsx')
df_sample_lang_article[10000:15000].to_excel('df_sample_lang_article_translation_file3.xlsx')
df_sample_lang_article[15000:len(df_sample_lang_article)].to_excel('df_sample_lang_article_translation_file4.xlsx')

In [ ]:
# translated version
temp1 = pd.read_excel('translation/df_sample_lang_article_translation_file1.xlsx')
temp2 = pd.read_excel('translation/df_sample_lang_article_translation_file2.xlsx')
temp3 = pd.read_excel('translation/df_sample_lang_article_translation_file3.xlsx')
temp4 = pd.read_excel('translation/df_sample_lang_article_translation_file4.xlsx')

In [ ]:
df_translation = pd.concat([temp1, temp2, temp3, temp4])

In [ ]:
import numpy as np

df_translation['article_text2'] = np.where(df_translation['article_text_translated'].isnull(), 
                                           df_translation['article_text'],
                                           df_translation['article_text_translated'])

In [ ]:
df_translation.columns

Index(['source', 'LANGUAGE', 'website', 'article_url', 'article_title',
       'article_author', 'date_published', 'article_text',
       'article_text_translated', 'LANGUAGE.1', 'article_text2'],
      dtype='object')

In [ ]:
test = df_translation.groupby(['source','LANGUAGE'])['article_text_translated'].count().reset_index()


In [ ]:
test[test['LANGUAGE']=='es']['article_text_translated'].sum()

3404

In [ ]:
test[test['LANGUAGE']=='fil']['article_text_translated'].sum()

1

In [ ]:
test[test['LANGUAGE']=='ko']['article_text_translated'].sum()

803

In [ ]:
len(test[test['LANGUAGE']=="es"])

17

In [ ]:
df_sample_lang_list = list(set(df_sample_lang.source.values.tolist()))
df_source_list = list(set(df_nna.source.values.tolist()))


In [ ]:
df_source_list_en = [i for i in df_source_list if i not in df_sample_lang_list]

In [ ]:
print(len(df_source_list_en))
print(len(df_source_list))
print(len(df_sample_lang_list))

212
231
19


In [ ]:
df_source_list_en_only = pd.DataFrame(df_source_list_en, columns = ["source"])

In [ ]:
df_source_list_en_only['en_source'] = 1

In [ ]:
df_source_list_en_only = pd.merge(df_source_list_en_only, df_nna, how="left", on="source")

In [ ]:
df_translation_done = df_translation[['source', 'website', 'article_url', 'article_title',
       'article_author', 'date_published', 'article_text2']]


In [ ]:
df_source_merge = pd.concat([df_source_list_en_only, df_translation_done])

In [ ]:
print(len(df_nna))
print(len(df_source_merge))

192206
192206


In [ ]:
import numpy as np

df_source_merge['article_final'] = np.where(df_source_merge['article_text2'].isnull(), df_source_merge['article_text'], df_source_merge['article_text2'])

In [ ]:
df_source_merge['article_final'].isnull().sum()

0

In [ ]:
df_source_merge_final = df_source_merge.drop(columns = ['article_text', 'article_text2'])

In [ ]:
df_source_merge_final.to_pickle('Georgia_data_v2_translation.pkl')

# Location analysis

## Prep for place analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis

In [ ]:
import pandas as pd

df_source_merge_final = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
# lemmatization if needed
import spacy
from spacy import displacy

sp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:

def text_ner_place(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if entity.label_ == "FAC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "GPE":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    if entity.label_ == "LOC":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return str(set(ner_list))


In [ ]:
df_source_merge_final['ner_places'] = df_source_merge_final['article_final'].astype(str).apply(text_ner_place)

In [ ]:
#df_source_merge_final.to_pickle('Georgia_data_v2_translation_ner.pkl') # commented this line to prevent from saving it

## Find rough location from spaCy

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
df_ner = df.loc[df['ner_places'] != 'set()']

In [ ]:
print(len(df))
print(len(df_ner))

192206
144929


In [ ]:
df_expand = df_ner['ner_places'].str.split(',').explode().str.strip('{} ')


In [ ]:
df_expand = pd.DataFrame(df_expand)


In [ ]:
df_expand['ner_places_clean'] = df_expand['ner_places'].str.replace(r"\'","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r" \"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\"","")
df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\(.*\)","")

<ipython-input-8-03789092516e>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places'].str.replace(r"\'","")
<ipython-input-8-03789092516e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r" \"","")
<ipython-input-8-03789092516e>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\"","")
<ipython-input-8-03789092516e>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_expand['ner_places_clean'] = df_expand['ner_places_clean'].str.replace(r"\(.*\)","")


In [ ]:
df_expand_count = df_expand.groupby('ner_places_clean').count().reset_index()

In [ ]:
df_freq_n5 = df_expand_count[df_expand_count["ner_places"] >= 5]
df_freq_n5.to_excel('df_freq_n5_ner_v2.xlsx')

In [ ]:
# # compare place entities with v1

# df_freq_n5_v2 = pd.read_excel('df_freq_n5_ner_v2.xlsx')
# df_freq_n5_v1 = pd.read_excel('df_freq_n5_ner.xlsx')

# df_freq_n5_v1['version'] = 'v1'
# df_freq_n5_v2['version'] = 'v2'

# df_freq_n5_merge = pd.merge( df_freq_n5_v1,df_freq_n5_v2, how="outer", on=["ner_places_clean"])

## Find sentence example of location

In [ ]:
import pandas as pd

#ner_places = pd.read_csv('Georgia_ner_places.csv') # previous data
ner_places = pd.read_csv('Georgia_ner_places_v2.csv') # updated data

In [ ]:
ner_places['ner_places_clean'] = ner_places['ner_places_clean'].str.rstrip()

In [ ]:
ner_places_raw = list(set(ner_places['ner_places_clean'].tolist()))

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# df["id"] = df.index + 1
df_ner = df.loc[df['ner_places'] != 'set()']

In [ ]:

import numpy as np

df_ner['place_ng1'] = df_ner['article_final'].astype(str).apply(lambda x: [i for i in ner_places_raw if i in x])


<ipython-input-76-22b7966504e5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ner['place_ng1'] = df_ner['article_final'].astype(str).apply(lambda x: [i for i in ner_places_raw if i in x])


In [ ]:
text_df_uni_dict = df_ner[['id',  'place_ng1']]
text_df_uni_article = df_ner[['id', "article_final"]]


In [ ]:

sample_sentence = text_df_uni_dict['place_ng1'].astype(str).str.split(',').explode().str.strip('[] ')
sample_sentence_df = pd.DataFrame(sample_sentence)

In [ ]:
merge_uni = pd.merge(sample_sentence_df, text_df_uni_article, left_index=True, right_index=True)

merge_uni['place_ng1'] = merge_uni['place_ng1'].str.strip('\'')

In [ ]:
merge_uni_count = merge_uni.groupby('place_ng1').count() #


In [ ]:
# articles mentioned a location less than 3
merge_uni_count_l3 = merge_uni_count[merge_uni_count['article_final'] < 3].reset_index()


In [ ]:
merge_uni_count_l3_places = merge_uni_count_l3.place_ng1.values.tolist()
merge_uni_count_l3_places

['Mountain Views',
 'St. Giles Cathedral',
 'St. Judes',
 'the State Fire Marshals Office']

In [ ]:
merge_uni_l3 = merge_uni[merge_uni['place_ng1'].isin(merge_uni_count_l3_places)]
merge_uni_l3_group = merge_uni_l3.groupby('place_ng1').apply(lambda x: x.sample(3, replace=True)).reset_index(drop=True)

In [ ]:
# articles mentioned a location more than 3 --> random sample

merge_uni_m3 = merge_uni[~merge_uni['place_ng1'].isin(merge_uni_count_l3_places)]
merge_uni_m3_group = merge_uni_m3.groupby('place_ng1').apply(lambda x: x.sample(3, replace=False)).reset_index(drop=True)


In [ ]:
# merge 3 samples + those with less than 3 
merge_uni_m3_group2 = pd.concat([merge_uni_m3_group, merge_uni_l3_group]).reset_index()

In [ ]:
merge_uni_m3_group2.rename(columns={'place_ng1':'places'}, inplace=True)


In [ ]:
merge_uni_m3_group2

,index,places,id,article_final
0,0,,158151,Panel recommends anxiety screening for adults ...
1,1,,39128,"This month, we’re recognizing a Springfield ed..."
2,2,,152966,"Watch CBS News\n""""Broadway in the Boros"""" brin..."
3,3,14 W. Main St.,47531,"GRAFTON — A new attraction described as a """"th..."
4,4,14 W. Main St.,30084,Gazebo Downtown Franklin decorated for Fall. p...
...,...,...,...,...
8284,7,St. Judes,173845,"September 23, 2022\nPublication: Northeast Geo..."
8285,8,St. Judes,173907,"By News Staff\nSeptember 23, 2022\nPublication..."
8286,9,the State Fire Marshals Office,102767,"SUMMERVILLE, S.C. (WCBD) – A deadly early morn..."
8287,10,the State Fire Marshals Office,102767,"SUMMERVILLE, S.C. (WCBD) – A deadly early morn..."


In [ ]:
# split an article into sentences

from nltk.tokenize import sent_tokenize
by_sentence = merge_uni_m3_group2['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


In [ ]:
by_sentence_df

,sentence_each,sentence_turn
0,Panel recommends anxiety screening for adults ...,0
0,The proposed measure intends to identify early...,1
0,"""""Screening all adults for depression, includi...",2
0,The panel includes generalized anxiety disorde...,3
0,"According to the panel, screening and follow-u...",4
...,...,...
8288,The cause of the fire is being investigated by...,12
8288,“It’s cooling off now.,13
8288,We’re going to go in and try to collect eviden...,14
8288,Editor’s Note: This is developing with updates...,15


In [ ]:
# merging split sentences to the article level
merge_all_m3_join = pd.concat([merge_uni_m3_group2, by_sentence_df], axis=1)


In [ ]:
merge_all_m3_join['search_words'] = merge_all_m3_join.apply(lambda x: x.places in x.sentence_each, axis=1)


In [ ]:
merge_all_m3_join['article_id'] = merge_all_m3_join.index.get_level_values(0)
merge_all_m3_join.reset_index(drop=True, inplace=True)


In [ ]:
# find those sentences that contain searching words (locations)

merge_all_m3_join_TRUE = merge_all_m3_join[merge_all_m3_join['search_words']==True]

In [ ]:
merge_temp = merge_all_m3_join_TRUE.groupby('places').count() #

,index,id,article_final,sentence_each,sentence_turn,search_words,article_id
places,,,,,,,
,22,22,22,22,22,22,22
14 W. Main St.,3,3,3,3,3,3,3
1495 Roswell Road,3,3,3,3,3,3,3
169/28,3,3,3,3,3,3,3
1:48,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...
uk,5,5,5,5,5,5,5
west Lincoln,3,3,3,3,3,3,3
west Tennessee,3,3,3,3,3,3,3


In [ ]:
# extract 3 random sentences containing the searching word (location) only

merge_temp_m3_group = merge_all_m3_join_TRUE.groupby('places').apply(lambda x: x.sample(3, replace=False)).reset_index(drop=True)


In [ ]:
merge_temp_m3_group

,index,places,id,article_final,sentence_each,sentence_turn,search_words,article_id
0,0,,158151,Panel recommends anxiety screening for adults ...,"According to the panel, screening and follow-u...",4,True,0
1,0,,158151,Panel recommends anxiety screening for adults ...,"In 2020, the National Alliance on Mental Illne...",10,True,0
2,0,,158151,Panel recommends anxiety screening for adults ...,"""""After diagnosis, people should participate i...",14,True,0
3,3,14 W. Main St.,47531,"GRAFTON — A new attraction described as a """"th...",The coaster ride is expected to be open year-r...,10,True,3
4,4,14 W. Main St.,30084,Gazebo Downtown Franklin decorated for Fall. p...,American Legion Macon Post 108 and American Le...,13,True,4
...,...,...,...,...,...,...,...,...
8254,8271,west entrance,48058,"UNI-Dome Render 1\n\nUpdated Oct 11, 2022\n\nA...","UNI-Dome Render 6\n\nUpdated Oct 11, 2022\n\nT...",8,True,8271
8255,8272,west entrance,47270,Iowa pork producers would suffer huge negative...,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...",36,True,8272
8256,8275,west.,92690,PHILADELPHIA — A local Republican Party in Sou...,"Now Richard Ringer, a former journalist, is ru...",54,True,8275
8257,8276,west.,105640,Darrell Brooks trial: Waukesha parade's younge...,He said he was concerned with 25 frames as the...,169,True,8276


In [ ]:
merge_temp_m3_group2 = merge_temp_m3_group[['places','id','sentence_each']]

In [ ]:
merge_temp_m3_group2['domainId'] = merge_temp_m3_group2.groupby('places')['places'].rank(method='first')


<ipython-input-114-32e3fd288afa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge_temp_m3_group2['domainId'] = merge_temp_m3_group2.groupby('places')['places'].rank(method='first')


In [ ]:
merge_temp_m3_group2.sort_values('places')

,places,id,sentence_each,domainId
0,,158151,"According to the panel, screening and follow-u...",1.0
1,,158151,"In 2020, the National Alliance on Mental Illne...",2.0
2,,158151,"""""After diagnosis, people should participate i...",3.0
3,14 W. Main St.,47531,The coaster ride is expected to be open year-r...,1.0
4,14 W. Main St.,30084,American Legion Macon Post 108 and American Le...,2.0
...,...,...,...,...
8253,west entrance,48058,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...",1.0
8255,west entrance,47270,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...",3.0
8257,west.,105640,He said he was concerned with 25 frames as the...,2.0
8256,west.,92690,"Now Richard Ringer, a former journalist, is ru...",1.0


In [ ]:
merge_temp_m3_group2_pivot = merge_temp_m3_group2.pivot(index='places', columns = 'domainId', values='sentence_each').reset_index()


In [ ]:
merge_temp_m3_group2_pivot.rename(columns={1:'sent_1', 2:'sent_2', 3:'sent_3'}, inplace=True)
merge_temp_m3_group2_pivot


domainId,places,sent_1,sent_2,sent_3
0,,"According to the panel, screening and follow-u...","In 2020, the National Alliance on Mental Illne...","""""After diagnosis, people should participate i..."
1,14 W. Main St.,The coaster ride is expected to be open year-r...,American Legion Macon Post 108 and American Le...,The coaster ride is expected to be open year-r...
2,1495 Roswell Road,"According to investigators, around 9:15 p.m. o...","According to police, around 9:15 p.m. Saturday...",Marietta PD’s Selective Traffic Enforcement Pr...
3,169/28,"Box: Troy Buchanan 44, Timberland 12\n\nUpdate...",1 2 3 4 Final\nVirden North Mac 15 7 14 14 50\...,"Box: Troy Buchanan 44, Timberland 12\n\nUpdate..."
4,1:48,Biden creates national monument in Colorado - ...,South Korea’s Joint Chiefs of Staff said in a ...,South Korea’s Joint Chiefs of Staff said in a ...
...,...,...,...,...
2748,uk,"The trio is one shot ahead of Ken Duke, who po...",“The DPRK clearly deems nuclear tests essentia...,"Senate Minority Leader Mitch McConnell, R-Ky.,..."
2749,west Lincoln,Police find 10 spent shell casings after gunsh...,Police find 10 spent shell casings after gunsh...,Police find 10 spent shell casings after gunsh...
2750,west Tennessee,A west Tennessee woman's accused of murdering ...,A west Tennessee woman's accused of murdering ...,A west Tennessee woman's accused of murdering ...
2751,west entrance,"UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA...","UNI-Dome Render 6\n\nUpdated Oct 11, 2022\n\nT...","UNI-Dome Render 3\n\nUpdated Oct 11, 2022\n\nA..."


In [ ]:
ner_places['places'] = ner_places['ner_places_clean']

In [ ]:
merge_temp_final = pd.merge(ner_places, merge_temp_m3_group2_pivot, on="places", how='left').reset_index()


In [ ]:
merge_temp_final['sent_1'] = merge_temp_final['sent_1'].str.replace('\n', ' ')
merge_temp_final['sent_2'] = merge_temp_final['sent_2'].str.replace('\n', ' ')
merge_temp_final['sent_3'] = merge_temp_final['sent_3'].str.replace('\n', ' ')

In [ ]:
# Mt. Pisgah
#merge_temp_final.to_excel('merge_temp_final.xlsx')
merge_temp_final.to_excel('/content/drive/MyDrive/Georgia/Locations Analysis/Georgia_ner_places_v2_wsent.xlsx')

### missing values


In [ ]:
# missing values

merge_temp_final_na = merge_temp_final[merge_temp_final['sent_1'].isnull()]


In [ ]:

missing_words = merge_temp_final_na.ner_places_clean.values.tolist()
len(missing_words)
                       

125

In [ ]:
df_ner['place_na'] = df_ner['article_final'].astype(str).apply(lambda x: [i for i in missing_words if i in x])


<ipython-input-112-1a733eefd377>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ner['place_na'] = df_ner['article_final'].astype(str).apply(lambda x: [i for i in missing_words if i in x])


In [ ]:
df_ner_place_na = df_ner[ df_ner['place_na'].str.len() > 0]

In [ ]:
df_ner_place_na 

,source,website,article_url,article_title,article_author,date_published,ner_places,article_lemma_num,article_final,id,place_ng1,place_na
175423,Appen Media,https://www.appenmedia.com,https://www.appenmedia.com/community/northern-...,Northern Ridge announces August Eagle Scouts,NaN,2022-09-29,"{'the Dunwoody Nature Center (FAC)', 'St. Bene...",northern ridge boy scout district proud announ...,The Northern Ridge Boy Scout District is proud...,175424,"[Center, Alpharetta, Alpha, St., Dunwoody, Park]",[Mt. Pisgah]
201713,Athens Banner Herald,http://onlineathens.com/,https://www.onlineathens.com/story/opinion/202...,Loran Smith: Stroll through campus brings back...,Loran Smith,2022-10-14,"{'Mt. Everest (LOC)', 'Georgia (GPE)', 'Dawgs ...",inconvenience good push frustration serendipit...,"Sometimes when you are inconvenienced, it is b...",201714,"[Street, Field Street, Camp, Sanford, Sanford ...",[Mt. Everest]
95384,Atlanta Journal-Constitution,http://www.ajc.com/,https://www.ajc.com/news/crime/commission-comp...,Commission completes work to rename Fort Benni...,Jozsef Papp,2022-09-20,"{'Fort Liberty (GPE)', 'Fort Walker (GPE)', 'V...",finish time spend quarter money authorize cong...,"“We finished on time, spending only a quarter ...",95385,"[Alabama, North Carolina, Texas, Fort Walker, ...",[Fort A.P. Hill]
94944,Atlanta Journal-Constitution,http://www.ajc.com/,https://www.ajc.com/sports/high-school-sports-...,"Class 4A: Week 5 Primer, Region 4 takes center...",Craig Sager II,2022-09-15,"{'Cairo (GPE)', 'Wayne County (GPE)', 'Clarkst...",limit opponent seven maybe possession stop tim...,"“If we can limit an opponent to seven, maybe e...",94945,"[Cherokee, Oconee, Richmond Hill, Grove, Walnu...","[Mt. Zion, Mt. Zion-Jonesboro]"
95162,Atlanta Journal-Constitution,http://www.ajc.com/,https://www.ajc.com/sports/high-school/halt-ke...,Halt: Kennesaw Mt. Paran Christian pushes the ...,Sports Bot,2022-09-17,"{'Kennesaw (GPE)', 'Mt. Paran (LOC)'}",eagle register 34 0 advantage intermission war...,The Eagles registered a 34-0 advantage at inte...,95163,"[Kennesaw, Mt. Paran]",[Mt. Paran]
...,...,...,...,...,...,...,...,...,...,...,...,...
2671,MundoNow,https://mundonow.com,https://mundonow.com/hombre-roba-motocicleta-2...,Hombre roba motocicleta: tan solo 20 segundos ...,Ernesto Iván Vargas Martínez,2022-09-21,"{'Tiktok (GPE)', 'Latin America (LOC)', 'Colón...",man steal motorcycle exhibit thief break glass...,Man steals a motorcycle that was being exhibit...,2672,"[La, Argentina, America, Córdoba, Latin Americ...",[Córdoba]
2721,MundoNow,https://mundonow.com,https://mundonow.com/imagenes-sismo-michoacan/,Imágenes sismo Michoacán de madrugada se sient...,Cuauhtemoc Torres,2022-09-22,"{'Michoacán (GPE)', 'Colima (GPE)', 'Jalisco (...",impressive image earthquake circulate michoacn...,Impressive images of the earthquake circulate ...,2722,"[States, Zacatecas, the United States, Mexico ...",[Zacatecas]
2734,MundoNow,https://mundonow.com,https://mundonow.com/muertos-sismo-68-en-mexico/,Confirman dos muertos tras sismo de magnitud 6...,MundoNOW,2022-09-22,"{'Michoacán (GPE)', 'Earth (LOC)', 'Colima (GP...",earthquake mexico leave dead 387 million peopl...,Earthquake in Mexico leaves at least two dead ...,2735,"[States, Zacatecas, the United States, Mexico ...",[Zacatecas]
2898,MundoNow,https://mundonow.com,https://mundonow.com/mhoni-vidente-sismos-mexico/,Mhoni Vidente sismos México predicciones era d...,Cuauhtemoc Torres,2022-09-27,"{'Chile (GPE)', 'Michoacán (GPE)', 'Oaxaca (GP...",mhoni seer make prediction earthquake mexico s...,Mhoni seer makes predictions on earthquake in ...,2899,"[States, Papua New Guinea, Mexico City, Mexico...",[Coalcomán]


## Look up location (w updated data) on articles

In [ ]:
%cd /content/drive/MyDrive/Georgia/Locations Analysis

In [ ]:
# data created by manual coding

import pandas as pd

location_final = pd.read_excel('Georgia_place_validation_counties.xlsx')

In [ ]:
location_final = location_final[location_final['final_selection']==1]

In [ ]:
location_final_st = location_final[['ner_place_clean','county_level', 'final_selection']]

In [ ]:
location_final_st = location_final_st.dropna()

In [ ]:
len(location_final_st)

1930

In [ ]:
location_final_st

,ner_place_clean,county_level,final_selection
0,110 Airport Road,Habersham County,1
1,1495 Roswell Road,Cobb County,1
2,26th Street,Fulton County,1
3,3227 Roswell Rd NE,Fulton County,1
4,46 Plaza Way,Rabun County,1
...,...,...,...
1950,Yonah Memorial Gardens,Habersham County,1
1951,York,Rabun County,1
1952,Youngstown,Union County,1
1953,Ypsilanti,Talbot County,1


In [ ]:
# # multiple counties deleted
# location_final_st = location_final_st[location_final_st['county_level'].str.contains('Counties')==False]

In [ ]:
len(set(location_final_st.county_level.values.tolist()))

159

In [ ]:
location_final_list = list(set(location_final_st['ner_place_clean'].tolist()))

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]

word_bigrams_done = list(set(bigrams(location_final_list)))
print(word_bigrams_done)
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(location_final_list)
print(word_unigrams_done)
print(len(word_unigrams_done))


['Mirror Lake Connector', 'North Green Street', 'Cobb DOT road', 'Greene County High School', 'Sugarloaf Parkway', 'the UGA Bookstore', 'Ridge Ferry Park', 'Meriweather County', 'Chestnut Drive', 'Peachtree Ridge', 'Grindstone Drive', 'the Fox Theatre', 'Statesboro Regional Library', 'West Armuchee', 'Covington Square', 'the Hartwell Library', 'West Georgia Tech', 'the Garden City Police Department', 'Capitol Avenue', 'Roosevelt Boulevard', 'Sandy Creek Park', 'The Classic Center', 'Rabbit Valley', 'Mirror Lake', 'Cobb Pkwy NW', 'Forest Lake', 'the Georgia Diagnostic and', 'Bobby Dodd Stadium', 'Wilder Center', 'the Wardlaw-Bible Meeting Room of the LaFayette-Walker County Library', 'Memorial Parkway', 'Kennesaw Memorial Park Cemetery', 'the Barrow County Detention Center', 'Old Mountain Road', 'Silver Creek', 'St. Charles', 'Lake Trahlyta', 'West Cobb Funeral Home and', 'Pearl Street', 'Wooten Lake Road', 'West Cobb Funeral Home', 'Bartow in', 'Gordon County’s', 'the White City', 'Fiv

In [ ]:

import numpy as np

df['place_ng1'] = df['article_final'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['place_ng2'] = df['article_final'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:
df_uni = df[(df['place_ng1'].str.len()) >= 1]
df_bi = df[(df['place_ng2'].str.len()) >= 1]

df_uni = df_uni[['id', 'source', 'place_ng1']]
df_bi = df_bi[['id', 'source', 'place_ng2']]

df_uni_df = df_uni.explode('place_ng1')
df_bi_df = df_bi.explode('place_ng2')

df_uni_df = df_uni_df.rename(columns={'place_ng1': 'place_ner'})
df_bi_df = df_bi_df.rename(columns={'place_ng2': 'place_ner'})


In [ ]:
df_ng_all = pd.concat([df_uni_df, df_bi_df])

In [ ]:
count = df_ng_all.groupby(['source']).count()

In [ ]:
count_20 = count[count['place_ner'] >= 20].reset_index()

In [ ]:
count_20_id = list(set(count_20['source'].tolist()))

df_ng_all_m20 = df_ng_all[df_ng_all['source'].isin(count_20_id)]


In [ ]:
len(count_20_id)

198

In [ ]:
location_final_st = location_final_st.rename(columns={'ner_place_clean': 'place_ner'})


In [ ]:
df_ng_all_m20_county = pd.merge(df_ng_all_m20, location_final_st, how='left', on='place_ner')

In [ ]:
df_ng_all_m20_county

,id,source,place_ner,county_level,final_selection
0,3,Metter Advertiser,Charles,Toombs County,1
1,3,Metter Advertiser,Franklin,Heard County,1
2,3,Metter Advertiser,Metter,Candler County,1
3,3,Metter Advertiser,William,Chatham County,1
4,3,Metter Advertiser,Murphy,Colquitt County,1
...,...,...,...,...,...
194285,191910,WALB TV 10 (NBC),Osprey Cove Drive,Muscogee County,1
194286,191944,WALB TV 10 (NBC),Magnolia Drive North,Effingham County,1
194287,191955,WALB TV 10 (NBC),Pine Street,Fulton County,1
194288,191985,WALB TV 10 (NBC),Wake County,Walker County,1


In [ ]:
df_ng_all_m20_county_unique = df_ng_all_m20_county.drop_duplicates(subset=['id', 'source', 'county_level'])

In [ ]:
df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique.groupby(['source','county_level'])['id'].count().reset_index()

In [ ]:
df_ng_all_m20_county_unique_count = df_ng_all_m20_county_unique_count.rename(columns={'id': 'article_id_count'})


In [ ]:
df_ng_all_m20_county_unique_count.to_excel('Georgia_m20_county_unique_count_v2.xlsx')

# Topic analysis

## Prep for topic analysis

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries


In [ ]:
# lemmatization if needed
import spacy
import re

nlp = spacy.load("en_core_web_sm", disable=['ner', 'parser']) # disabling Named Entity Recognition for speed


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import re

def get_lemmas_num(text):

    regex = re.compile('[^a-zA-Z0-9]')

    lemmas = []
    
    doc = nlp(text)

    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON') and (token.text != 'NA'):
            lemmas.append(regex.sub('', token.lemma_))
    
    return ' '.join(lemmas)

In [ ]:
df_source_merge_final['article_lemma_num'] = df_source_merge_final['article_final'].str.lower().astype(str).apply(get_lemmas_num)


In [ ]:
#df_source_merge_final.to_pickle('Georgia_data_v2_translation_ner_lem.pkl') # commented this line to prevent from saving it

## word2vec training

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
import pandas as pd
# To extract n-gram from text
from gensim.models.phrases import Phrases, Phraser
# To train word2vec
from gensim.models import Word2Vec
# To load pre trained word2vec
from gensim.models import KeyedVectors


In [ ]:
df.columns

Index(['source', 'en_source', 'website', 'article_url', 'article_title',
       'article_author', 'date_published', 'article_final', 'ner_places',
       'ner_orgs', 'article_lemma_num'],
      dtype='object')

In [ ]:
# As Phrases() takes list of list as it's input
# Converting dataframe to list of list

## all lower cases

sentence_stream = [doc.split(" ") for doc in df['article_lemma_num']]
bigram = Phrases(sentence_stream, min_count= 10, delimiter=b' ') # v2 threshold as default
bigram_phraser = Phraser(bigram)

# bigram = Phrases(sentence_stream, min_count=10, threshold= 10, delimiter=b' ') # higher threshold fewer phrases.
# trigram = Phrases(bigram[sentence_stream], threshold= 100, delimiter=b' ') # include bi- and tri-grams


# Extract bigram for gensim word2vec
bigram_token = []

for sent in sentence_stream:
    bigram_token.append(bigram_phraser[sent])

article_model_bigram = Word2Vec(bigram_token, min_count=10, size= 40, workers=5, window = 5, sg = 0)
article_model_bigram.save("Georgia_language_lemma_num10_v2") # v2 : with updated dataset


## topic words identification

### separate analysis for words with asterisk

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries


In [ ]:
topics = pd.read_excel("topics.xlsx")

topics['word_lemma'] = topics['word'].str.lower().astype(str).apply(get_lemmas_num)


In [ ]:
import numpy as np

star_phrases = topics['word'].str.contains('\*{1}', case=False, regex=True)

topics['word_star'] = np.where(star_phrases, 1, 0)


In [ ]:
topics_st = topics[['topic', 'word_lemma', 'word_star']].drop_duplicates()
topics_st.sample(3)

,topic,word_lemma,word_star
376,Education,kindergarten,0
411,Crime,assault,0
250,Climate change,endanger specie,0


In [ ]:
topics_st_asterisk = topics_st[topics_st['word_star']==1]

In [ ]:
topics_st_asterisk_list = topics_st_asterisk['word_lemma'].values.tolist()

In [ ]:
words = pd.DataFrame(list(article_model_bigram.wv.vocab.keys()))
words.rename(columns={0:'n_grams'}, inplace=True)


In [ ]:
words['word_lemma'] = words['n_grams'].astype(str).apply(lambda x: [i for i in topics_st_asterisk_list if i in x])


In [ ]:
words_filter = words[ words['word_lemma'].str.len() > 0 ]

In [ ]:
words_filter_unnest = words_filter.explode('word_lemma')


In [ ]:
words_filter_unnest = pd.merge(words_filter_unnest, topics_st_asterisk, on="word_lemma", how='left')

In [ ]:
words_filter_unnest.to_excel('additional_topic_words_v2.xlsx')

In [ ]:
additional_topic_words_v2 = pd.read_excel('additional_topic_words_v2.xlsx')
additional_topic_words_v1 = pd.read_excel('additional_topic_words.xlsx')

In [ ]:
additional_topic_words_v1['version'] = 'v1'
additional_topic_words_v2['version'] = 'v2'

add_topic_words_merge = pd.merge( additional_topic_words_v1, additional_topic_words_v2, how="outer", on=["n_grams", "topic"])

In [ ]:
add_topic_words_merge = add_topic_words_merge[['n_grams', 'topic', 'FINAL', 'version_x', 'version_y']]

In [ ]:
add_topic_words_merge.to_excel('Topic_words_review1_v1_v2_diff.xlsx')


In [ ]:
#add_topic_words_merge_NA = add_topic_words_merge[ (add_topic_words_merge['version_x'].isnull()) | (add_topic_words_merge['version_y'].isnull()) ]


### final list prep

In [ ]:
add_words = pd.read_excel("/content/drive/MyDrive/Georgia/Topic Dictionaries/Topic_words_review1_v1_v2_diff.xlsx")

In [ ]:
topics_R1 = topics_st[topics_st['word_star'] == 0]
topics_R1 = topics_R1[['topic', 'word_lemma']]

In [ ]:
topics_add = add_words[add_words['FINAL2'] == 1]
topics_add = topics_add[['topic', 'n_grams']]
topics_add.rename(columns={'n_grams': 'word_lemma'}, inplace=True)


In [ ]:
topic_final = pd.concat([topics_R1, topics_add])

In [ ]:
topic_final = topic_final.drop_duplicates()

In [ ]:
topic_final

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2848,Racial inequality,minority
2858,Racial inequality,nondiscrimination
2882,Racial inequality,stigma
2890,Racial inequality,stigmatize


## topic words similarity

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import gensim

article_model_bigram = gensim.models.Word2Vec.load("Georgia_language_lemma_num10_v2")


In [ ]:
topic_dictionary = list(set(topic_final.word_lemma.tolist()))

In [ ]:
## update this code to create a dataset with topic words; do it again among the lsd words

word_list_scores = []

for topic_word in topic_dictionary:
  try:
    word_list_scores.append( (topic_word, article_model_bigram.wv.most_similar(topic_word, topn=10)) )
  except KeyError:
    word_list_scores.append( (topic_word, 'NA') )

In [ ]:
df_test = pd.DataFrame ( word_list_scores )


In [ ]:
df_test.rename(columns={0: 'word_lemma', 1: 'similar_words'}, inplace=True)

df_text_spread = df_test.explode('similar_words')

In [ ]:
# df_text_spread_NA = df_text_spread[df_text_spread['similar_words']=='NA']
# df_text_spread_NA.to_excel('FINA_topic_words_not_include_review.xlsx')

In [ ]:
df_text_spread = df_text_spread[df_text_spread['similar_words']!='NA']
df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]
df_text_spread = df_text_spread.drop('similar_words', axis=1)

<ipython-input-80-b3f5d5179380>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text_spread['similar_word'], df_text_spread['similarity'] = df_text_spread.similar_words.str[0], df_text_spread.similar_words.str[1]


In [ ]:
df_text_spread_merge = pd.merge(df_text_spread, topic_final, how='left', on='word_lemma')

In [ ]:
# those with higher than 0.7 similarity

df_test_sim07 = df_text_spread_merge[ df_text_spread_merge['similarity'] > 0.7  ]

In [ ]:
# 4681 previous
# 4391 current

In [ ]:
df_test_sim07 # 4681

,word_lemma,similar_word,similarity,topic
10,recycle,compost,0.844696,Climate change
11,recycle,reusable,0.805904,Climate change
12,recycle,container,0.798560,Climate change
13,recycle,reuse,0.794733,Climate change
14,recycle,recyclable,0.794667,Climate change
...,...,...,...,...
5105,salary,retention bonus,0.715568,Economy / Jobs
5106,salary,wage,0.714138,Economy / Jobs
5107,salary,pension,0.713663,Economy / Jobs
5108,salary,earning,0.710439,Economy / Jobs


In [ ]:
df_test_sim07_black = df_test_sim07[df_test_sim07['word_lemma']=='black']
df_test_sim07_black

,word_lemma,similar_word,similarity,topic
4420,black,white,0.844675,Racial inequality
4421,black,color,0.715145,Racial inequality


In [ ]:
df_test_sim07_nondup = df_test_sim07[['similar_word', 'topic']].drop_duplicates()

In [ ]:
df_test_sim07_nondup = df_test_sim07.drop_duplicates(subset=['similar_word', 'topic'])

In [ ]:
df_test_sim07_nondup # 3289 non-duplicates --> 3145

,word_lemma,similar_word,similarity,topic
10,recycle,compost,0.844696,Climate change
11,recycle,reusable,0.805904,Climate change
12,recycle,container,0.798560,Climate change
13,recycle,reuse,0.794733,Climate change
14,recycle,recyclable,0.794667,Climate change
...,...,...,...,...
5095,vaccinated,prevalence,0.778021,COVID-19
5096,vaccinated,severe illness,0.774695,COVID-19
5097,vaccinated,virus cause,0.773555,COVID-19
5098,vaccinated,syndrome mutation,0.771601,COVID-19


In [ ]:
# exclude those from the topic dictionary
df_test_sim07_final = df_test_sim07_nondup[~df_test_sim07_nondup['similar_word'].isin(topic_dictionary)]


In [ ]:
df_test_sim07_final.to_excel('df_test_sim07_top10_no_duplicates2_v2.xlsx')

In [ ]:
df_test_sim07_top10_v1 = pd.read_excel('df_test_sim07_top10_no_duplicates2.xlsx')
df_test_sim07_top10_v2 = pd.read_excel('df_test_sim07_top10_no_duplicates2_v2.xlsx')



In [ ]:
df_test_sim07_top10_v1 = df_test_sim07_top10_v1[['similar_word', 'topic', 'FINAL']]

In [ ]:
df_test_sim07_top10_v2 = df_test_sim07_top10_v2[['similar_word', 'topic']]

In [ ]:
df_test_sim07_top10_v2['version'] = 'v2'

<ipython-input-145-1b2dc470a668>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_sim07_top10_v2['version'] = 'v2'


In [ ]:
df_sim07_top10_merge = pd.merge( df_test_sim07_top10_v1, df_test_sim07_top10_v2, how="outer", on=["similar_word", "topic"])

In [ ]:
df_sim07_top10_merge[df_sim07_top10_merge['version'].isnull()]

,similar_word,topic,FINAL,version
2,exception,Abortion,0.0,NaN
4,conservative,Abortion,0.0,NaN
5,expand electorate,Abortion,0.0,NaN
6,fellow democrat,Abortion,0.0,NaN
7,gop candidate,Abortion,0.0,NaN
...,...,...,...,...
2886,woke,Racial inequality,0.0,NaN
2891,court decision,Supreme Court,0.0,NaN
2892,court ruling,Supreme Court,0.0,NaN
2893,dobb,Supreme Court,0.0,NaN


In [ ]:
df_sim07_top10_merge[df_sim07_top10_merge['version']=='v2']

,similar_word,topic,FINAL,version
0,abortion ban,Abortion,0.0,v2
1,ban abortion,Abortion,0.0,v2
3,oppose abortion,Abortion,0.0,v2
12,choose,Abortion,0.0,v2
13,prefer,Abortion,0.0,v2
...,...,...,...,...
4545,prevalence,COVID-19,NaN,v2
4546,severe illness,COVID-19,NaN,v2
4547,virus cause,COVID-19,NaN,v2
4548,syndrome mutation,COVID-19,NaN,v2


In [ ]:
df_sim07_top10_merge.to_excel('df_sim07_top10_merge_v1_v2_diff.xlsx')

In [ ]:
topic_final_r1 = topic_final.copy()
topic_final_r1['topic_final'] = 1

In [ ]:
topic_final_r1.rename(columns={'word_lemma': 'similar_word'}, inplace=True)


In [ ]:
df_sim07_top10_merge_all = pd.merge(df_sim07_top10_merge, topic_final_r1, how="outer", on=["similar_word", "topic"])

In [ ]:
df_sim07_top10_merge_all['review_done'] = np.where(  (df_sim07_top10_merge_all['FINAL']== 0) & (df_sim07_top10_merge_all['version']== "v2"), 1, 0  )


In [ ]:
df_sim07_top10_merge_all.to_excel('df_sim07_top10_merge_v1_v2_diff.xlsx')

## final topic word list

In [ ]:
import pandas as pd
topic_R2 = pd.read_excel('df_test_sim07_top10_no_duplicates2.xlsx')

In [ ]:
topic_R2 = topic_R2[topic_R2['FINAL'] == 1]
topic_R2 = topic_R2[['topic', 'similar_word']]
topic_R2.rename(columns={'similar_word': 'word_lemma'}, inplace=True)


In [ ]:
topic_R2_add = pd.read_excel('additional_words_manual.xlsx') 
topic_R2_add.rename(columns={'similar_word': 'word_lemma'}, inplace=True)


In [ ]:
topic_final

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
2082,Racial inequality,marginalize
2085,Racial inequality,marginalized
2092,Racial inequality,minority
2107,Racial inequality,stigma


In [ ]:
topic_final[topic_final['word_lemma']=='undocumente']

,topic,word_lemma
122,Immigration / border security,undocumente


In [ ]:
topic_final2 = pd.concat([topic_final, topic_R2, topic_R2_add])

In [ ]:
topic_final2

,topic,word_lemma
0,Inflation,cost live
1,Inflation,inflation
3,Inflation,afford
4,Inflation,affordability
5,Inflation,affordable
...,...,...
0,Abortion,prochoice
1,Abortion,antichoice
2,Abortion,dobbs
3,Crime,gunfire


In [ ]:
len(topic_final2)

940

In [ ]:
topic_final2 = topic_final2.drop_duplicates().sort_values(['topic','word_lemma']).reset_index()

In [ ]:
topic_final2 = topic_final2[['topic','word_lemma']]

In [ ]:
# delete depression AND lay from Economy / Jobs	
# delete traffic from Immigration / border security AND Crime

In [ ]:
topic_final2[topic_final2['word_lemma'] == 'undocumented'] # 467


,topic,word_lemma
821,Immigration / border security,undocumented


In [ ]:
topic_final2[topic_final2['word_lemma'] == 'depression'] # 418
topic_final2[topic_final2['word_lemma'] == 'lay'] # 467
topic_final2[topic_final2['word_lemma'] == 'traffic'] # 371, 818
topic_final2[topic_final2['word_lemma'] == 'undocumente'] # 820

,topic,word_lemma
820,Immigration / border security,undocumente


In [ ]:
topic_final2[topic_final2['word_lemma'] == 'cap trade '] # 94

,topic,word_lemma
94,Climate change,cap trade


In [ ]:
topic_final2.drop([94, 418,467, 371, 818, 820], axis=0, inplace=True)


In [ ]:
topic_final2.to_excel('FINAL_topic_words_R2_review.xlsx')

## mturk review for v2

In [ ]:
%cd /content/drive/MyDrive/Georgia/Topic Dictionaries

import pandas as pd

review_v2 = pd.read_excel('/content/drive/MyDrive/Georgia/Topic Dictionaries/FINAL_sim07_top10_merge.xlsx')

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Topic Dictionaries


In [ ]:
review_v2_review = review_v2[ (review_v2['FINAL_ADD']==1)]

In [ ]:
review_v2_review_st = review_v2_review[['similar_word', 'topic']]

In [ ]:
len(review_v2_review_st)

317

## find example sentences

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
len(df[df['article_lemma_num'].isnull()])

0

### round 1

In [ ]:
topic_final2 = pd.read_excel('FINAL_topic_words_R2_review.xlsx')

In [ ]:
topic_final2.rename(columns={'word_lemma': 'topic_ng'}, inplace=True)


In [ ]:
topic_final2 = topic_final2[['topic','topic_ng']].drop_duplicates() # 933

In [ ]:
topic_final2 = topic_final2[topic_final2['topic_ng'] != 'choice'] # remove choice 


### round 2

In [ ]:
review_v2_review_st = review_v2_review[['similar_word', 'topic']]
review_v2_review_st.rename(columns={'similar_word': 'topic_ng'}, inplace=True)
review_v2_review_st = review_v2_review_st[['topic','topic_ng']].drop_duplicates() # 313

/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [ ]:
len(review_v2_review_st)

313

In [ ]:

import re

def bigrams(s):
    return [i for i in s if re.search(r'\s', i) ]
  
def unigrams(s):
    return [i for i in s if not re.search(r'\s', i) ]


topic_words = list(set(review_v2_review_st['topic_ng'].tolist()))

word_bigrams_done = list(set(bigrams(topic_words)))
print(word_bigrams_done)
print(len(word_bigrams_done))
                       
word_unigrams_done = unigrams(topic_words)
print(word_unigrams_done)
print(len(word_unigrams_done))

['gas emission', 'source methane', 'emergency contraception', 'taxable income', 'conspiracy commit', 'illegally enter', 'reverence life', 'growth slow', 'anxiety screening', 'request asylum', 'venous thromboembolism', 'foreign national', 'arm dealer', 'death penalty', 'delta variant', 'blood thinner', 'certify presidential', 'brittany fonteno', 'coral reef', 'supply shortage', 'fda approve', 'deficit reduction', 'bodily autonomy', 'enroll medicare', 'arm trafficking', 'accusation sexual', 'dismiss illegitimate', 'knee surgery', 'unplanned pregnancy', 'break in', 'abbott bus', 'legal pathway', 'corona 19', 'financial institution', 'sleep habit', 'equity diversity', 'racial disparity', 'aggravate battery', 'american academy', 'attempt overturn', 'charge aggravated', 'birth defect', 'cost effective', 'gang relate', 'cognitive impairment', 'arrest allegedly', 'southern border', 'wildfire smoke', 'stock index', 'hearing aid', 'consumer spending', 'border mexico', 'primary caregiver', 'stock

In [ ]:

import numpy as np

df['topic_ng1'] = df['article_lemma_num'].astype(str).apply(lambda x: set.intersection(set(x.split(' ')), word_unigrams_done))
df['topic_ng2'] = df['article_lemma_num'].astype(str).apply(lambda x: [i for i in word_bigrams_done if i in x])


In [ ]:
df.sample(3)

,source,website,article_url,article_title,article_author,date_published,ner_places,article_lemma_num,article_final,translation_YN,id,ner_orgs,topic_ng1,topic_ng2
30481,Polk County Standard Journal,http://www.northwestgeorgianews.com/polkfishwrap/,https://www.northwestgeorgianews.com/fieldleve...,"Bret Bielema faces old team, Illinois visits W...",Field Level Media,2022-09-26,"{'Italy (GPE)', 'Canada (GPE)', 'Republic of I...",country united states america virgin islands u...,Country United States of America US Virgin Isl...,0,30482,"{'Grand Duchy of Macao (ORG)', 'Republic of Bo...",{},[]
1796,Douglas County Sentinel,http://www.douglascountysentinel.com/,https://www.douglascountysentinel.com/lifestyl...,AI Could Improve Diagnostic Accuracy of Breast...,Physician’s Briefing Staff,2022-10-07,"{'New York City (GPE)', 'Ph.D. (GPE)', 'Poland...",friday oct 7 2022 healthday news deep learning...,"FRIDAY, Oct. 7, 2022 (HealthDay News) -- A dee...",0,1797,"{'Science Translational Medicine (ORG)', 'BI-R...","{cancer, mri}",[cancer diagnosis]
125048,WAGA TV 5 (Fox),http://www.myfoxatlanta.com/,https://www.fox5atlanta.com/news/shaw-universi...,"Shaw University students stopped, searched in ...",FOX 5 Atlanta Digital Team,2022-10-11,"{'Atlanta (GPE)', 'Raleigh (GPE)', 'North Caro...",article exterior shaw university locate raleig...,"article Exterior of Shaw University, located i...",0,125049,"{'Shaw University (ORG)', 'the Center for Fina...",{},[]


In [ ]:

text_df_uni = df[(df['topic_ng1'].str.len()) > 0]
text_df_bi = df[(df['topic_ng2'].str.len()) > 0]

text_df_uni = text_df_uni[['id', 'source', 'article_final', 'topic_ng1']]
text_df_bi = text_df_bi[['id', 'source','article_final', 'topic_ng2']]

text_df_uni_df = text_df_uni.explode('topic_ng1')
text_df_bi_df = text_df_bi.explode('topic_ng2')

text_df_uni_df = text_df_uni_df.rename(columns={'topic_ng1': 'topic_ng'})
text_df_bi_df = text_df_bi_df.rename(columns={'topic_ng2': 'topic_ng'})

text_df_ng_all = pd.concat([text_df_uni_df, text_df_bi_df])



In [ ]:
text_df_ng_all = pd.merge(text_df_ng_all, review_v2_review_st, how='left', on = 'topic_ng')

In [ ]:
text_df_ng_all # three words missing

,id,source,article_final,topic_ng,topic
0,173704,285 South,The GOP effort to court Hispanic voters: Repub...,shooting,Guns
1,173702,285 South,The Guatemalan population in Georgia has almos...,migrate,Immigration / border security
2,173703,285 South,It\u2019s a busy month - lots of fundraisers a...,business,Economy / Jobs
3,173705,285 South,"Sara Naqvi, publisher of the blog The Pakistan...",business,Economy / Jobs
4,173705,285 South,"Sara Naqvi, publisher of the blog The Pakistan...",immigrate,Immigration / border security
...,...,...,...,...,...
102757,3813,WQXI AM 790,"Mi, 3 consecutive Giant Step… Return to the RO...",fed raise,Economy / Jobs
102758,3813,WQXI AM 790,"Mi, 3 consecutive Giant Step… Return to the RO...",raise interest,Economy / Jobs
102759,3814,WQXI AM 790,"2022 Fiscal year, more than 2 million immigran...",migrant cross,Immigration / border security
102760,3815,WQXI AM 790,The Fed's Economic Analyst is a 0.75 point inc...,fed raise,Economy / Jobs


In [ ]:
temp_count = text_df_ng_all.groupby(['topic', 'topic_ng']).count().reset_index()

In [ ]:
temp_count

,topic,topic_ng,id,source,article_final
0,Abortion,access reproductive,41,41,41
1,Abortion,bodily autonomy,35,35,35
2,Abortion,brittany fonteno,10,10,10
3,Abortion,fetal,243,243,243
4,Abortion,life unborn,24,24,24
...,...,...,...,...,...
306,Racial inequality,practice disproportionately,30,30,30
307,Racial inequality,prejudicial,31,31,31
308,Racial inequality,protest racial,22,22,22
309,Racial inequality,racial disparity,60,60,60


In [ ]:
# merge_uni_m10 = text_df_ng_all[~text_df_ng_all['topic_ng'].isin(temp_count_l10_topic)]
# merge_uni_m10_group = merge_uni_m10.groupby('topic_ng').apply(lambda x: x.sample(10, replace=False)).reset_index(drop=True)

# merge_uni_l10 = text_df_ng_all[text_df_ng_all['topic_ng'].isin(temp_count_l10_topic)]
# merge_uni_l10_group = merge_uni_l10.groupby('topic_ng').apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)

# merge_uni_both = pd.concat([merge_uni_m10_group, merge_uni_l10_group])

merge_uni_both = text_df_ng_all.groupby(['topic', 'topic_ng']).apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)


In [ ]:
merge_uni_both

,id,source,article_final,topic_ng,topic
0,141601,Decaturish,Decaturish and Tucker Observer sent candidate ...,access reproductive,Abortion
1,182746,WRBL News 3 (CBS),(NerdWallet) – Would you be able to quickly ob...,access reproductive,Abortion
2,180022,WJBF TV 6 (ABC),(NerdWallet) – Would you be able to quickly ob...,access reproductive,Abortion
3,48707,WGCL TV 46 (CBS),WASHINGTON (AP) — With few options for strengt...,access reproductive,Abortion
4,68691,WSB AM 750,As abortion remains a key issue in the upcomin...,access reproductive,Abortion
...,...,...,...,...,...
3105,19927,Atlanta Black Star,The white woman who made headlines in 2020 for...,racialize,Racial inequality
3106,186815,WSB TV 2 (ABC),"A single pair of Levi’s jeans, dating to the 1...",racialize,Racial inequality
3107,67742,WRFC AM 960,"A single pair of Levi’s jeans, dating to the 1...",racialize,Racial inequality
3108,67616,WRFC AM 960,Tulsi Gabbard says she is leaving the Democrat...,racialize,Racial inequality


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
by_sentence = merge_uni_both['article_final'].apply(lambda x : sent_tokenize(x)).apply(pd.Series,1).stack()
by_sentence_df = pd.DataFrame(by_sentence)
by_sentence_df.rename(columns={0:'sentence_each'}, inplace=True)

by_sentence_df['sentence_turn'] = by_sentence_df.index.get_level_values(1)
by_sentence_df.index = by_sentence_df.index.get_level_values(0)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
merge_all_join = pd.concat([merge_uni_both, by_sentence_df], axis=1)
merge_all_join['sentence_each_lemma'] = merge_all_join['sentence_each'].str.lower().astype(str).apply(get_lemmas_num)



In [ ]:
merge_all_join['topic_words_YN'] = merge_all_join.apply(lambda x: x.topic_ng in x.sentence_each_lemma, axis=1)
merge_all_join['topic_words_YN_lead'] = merge_all_join.groupby(['topic', 'topic_ng'])['topic_words_YN'].shift(-1)
merge_all_join.reset_index(drop=True, inplace=True)



In [ ]:
merge_all_join["row_id"] = merge_all_join.index + 1


In [ ]:
merge_sentence_select_in = merge_all_join[ (merge_all_join['topic_words_YN']== True)]


In [ ]:
merge_sentence_only = merge_sentence_select_in.drop(columns=['article_final'])
merge_sentence_only['str_count'] = merge_sentence_only.sentence_each.str.count(" ") # remove string count less than 3

In [ ]:
merge_sentence_only = merge_sentence_only[merge_sentence_only['str_count'] >= 3]

In [ ]:
merge_sentence_only_10 = merge_sentence_only.groupby(['topic', 'topic_ng']).apply(lambda x: x.sample(10, replace=True)).reset_index(drop=True)


In [ ]:
test = merge_sentence_only_10.groupby(['topic', 'topic_ng']).count().reset_index()
test

,topic,topic_ng,id,source,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id,str_count
0,Abortion,access reproductive,10,10,10,10,10,10,10,10,10
1,Abortion,bodily autonomy,10,10,10,10,10,10,10,10,10
2,Abortion,brittany fonteno,10,10,10,10,10,10,10,10,10
3,Abortion,fetal,10,10,10,10,10,10,10,10,10
4,Abortion,life unborn,10,10,10,10,10,10,10,10,10
...,...,...,...,...,...,...,...,...,...,...,...
306,Racial inequality,practice disproportionately,10,10,10,10,10,10,10,10,10
307,Racial inequality,prejudicial,10,10,10,10,10,10,10,10,10
308,Racial inequality,protest racial,10,10,10,10,10,10,10,10,10
309,Racial inequality,racial disparity,10,10,10,10,10,10,10,10,10


In [ ]:
merge_sentence_only_10.sample(3)

,id,source,topic_ng,topic,sentence_each,sentence_turn,sentence_each_lemma,topic_words_YN,topic_words_YN_lead,row_id,str_count
1305,417,La Visión,price gasoline,Economy / Jobs,The prices of gasoline in the Metropolitan Are...,0,price gasoline metropolitan area atlanta shoot...,True,False,49776,29
1882,141132,Dalton Daily Citizen,cardiomyopathy,Health care,His mother said jailers refused to take him to...,14,mother say jailer refuse hospital medical care...,True,False,70731,19
636,26801,Lanier County News,armed dangerous,Crime,Jashun Wilkins is considered armed and dangerous.,14,jashun wilkin consider armed dangerous,True,False,25765,6


In [ ]:
merge_sentence_only_10_st = merge_sentence_only_10[['id','topic_ng','topic','sentence_each']]

In [ ]:
# # remove choice

# merge_sentence_only_10_st = merge_sentence_only_10_st[merge_sentence_only_10_st['topic_ng'] != 'choice']

In [ ]:
len(merge_sentence_only_10_st)

3110

In [ ]:
merge_sentence_only_10_st = merge_sentence_only_10_st.sample(len(merge_sentence_only_10_st))
merge_sentence_only_10_st.to_excel('Georgia_topic_words_validation_sentence10_v2.xlsx')

# Organization analysis

## spaCy -- pulling organizations only

In [ ]:
# lemmatization if needed
import spacy
from spacy import displacy

#sp = spacy.load('en_core_web_sm')
sp = spacy.load('en_core_web_sm', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])


def text_ner_org(text):
  text = sp(text)
  
  ner_list = []
  
  for entity in text.ents:
    if entity.label_ == "ORG":
      ner_list.append(entity.text + " (" + entity.label_ + ")")
    else:
      None
  return str(set(ner_list))

### org analysis

In [ ]:
import pandas as pd

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')


In [ ]:
df['ner_orgs'] = df['article_final'].astype(str).apply(text_ner_org)

In [ ]:
max(df['date_published'])

'2022-10-15'

In [ ]:
min(df['date_published'])

'2022-09-15'

In [ ]:
df_ner_comp_no_set = df[df['ner_orgs'] != 'set()']

In [ ]:
df_ner_comp_no_set

,index,source,website,article_url,article_title,article_author,date_published,ner_places,article_lemma_num,article_final,translation_YN,id,ner_orgs
0,173703,285 South,https://285south.bulletin.com/,https://285south.bulletin.com/voters-elections...,"Voters, elections, and voters.",Sophia Qureshi,2022-10-12,"{'Peachtree Corners (LOC)', 'Atlanta (GPE)', '...",gop effort court hispanic voter republican inc...,The GOP effort to court Hispanic voters: Repub...,0,1,"{'Latinos (ORG)', 'State Senate (ORG)', 'WABE ..."
1,173701,285 South,https://285south.bulletin.com/,https://285south.bulletin.com/as-the-guatemala...,"As the Guatemalan community in Georgia grows, ...",Sophia Qureshi,2022-09-28,"{'Guatemala (GPE)', 'Chuj (GPE)', 'Atlanta (GP...",guatemalan population georgia double 68000 dec...,The Guatemalan population in Georgia has almos...,0,2,"{'the Mayan Heritage Project (ORG)', 'Fundacio..."
2,173702,285 South,https://285south.bulletin.com/,https://285south.bulletin.com/events-around-to...,October Events Calendar,Sophia Qureshi,2022-09-30,"{'The City of Lawrenceville (GPE)', 'Johns Cre...",itu2019s busy month lot fundraiser event mark ...,It\u2019s a busy month - lots of fundraisers a...,0,3,"{'Noor Family Services Gala (ORG)', 'Labor (OR..."
3,173700,285 South,https://285south.bulletin.com/,https://285south.bulletin.com/victor-mariachi-...,Victor Mariachi didn't believe in voting. How ...,Sophia Qureshi,2022-09-16,"{'Phoenix (GPE)', 'Atlanta (GPE)', 'US (GPE)',...",285 south contributor phoebe quinton saturday ...,By 285 South Contributor Phoebe Quinton On Sat...,0,4,"{'LCF (ORG)', 'Latino (ORG)', 'Super Humans (O..."
4,173704,285 South,https://285south.bulletin.com/,https://285south.bulletin.com/meet-sara-naqvi-...,Meet Sara Naqvi: Pakistani vegan and co-found...,Sophia Qureshi,2022-10-14,"{'Atlanta (GPE)', '\\""""foodie\\ (GPE)', 'Georg...",sara naqvi publisher blog pakistani vegan peop...,"Sara Naqvi, publisher of the blog The Pakistan...",0,5,"{'Punjab Spice Company - a metro (ORG)', 'Rain..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
212992,3825,WQXI AM 790,https://atlantaradiokorea.com/,https://atlantaradiokorea.com/%ed%95%b4%ed%8a%...,"해트트릭으로 부활한 손흥민, EPL 공식 베스트11",뉴스투데이,2022-09-20,set(),million dollar evaporate land star usain bolt ...,Millions of dollars evaporated in the land sta...,1,212993,"{""Usain Bolt' (ORG)"", 'JAN (ORG)'}"
212993,3826,WQXI AM 790,https://atlantaradiokorea.com/,https://atlantaradiokorea.com/%ea%b2%80%ec%88%...,검수완박 권한쟁의 27일 공개변론…한동훈 직접 출석,Atlanta Radio Korea,2022-09-21,set(),atlanta radio korea atlanta radio korea fm967 ...,Atlanta Radio Korea Atlanta Radio Korea FM96.7...,1,212994,{'Atlanta Radio Korea (ORG)'}
212994,3827,WQXI AM 790,https://atlantaradiokorea.com/,https://atlantaradiokorea.com/%eb%af%b8-%ec%9e...,"미 재무부, '대북 석유수출' 개인 2명·사업체 3곳 제재",Atlanta Radio Korea,2022-10-07,set(),atlanta radio korea atlanta radio korea fm967 ...,Atlanta Radio Korea Atlanta Radio Korea FM96.7...,1,212995,{'Atlanta Radio Korea (ORG)'}
212995,3828,WQXI AM 790,https://atlantaradiokorea.com/,https://atlantaradiokorea.com/%ea%b7%80%eb%84%...,"귀넷, 폭주 및 레이싱 차량 처벌 강화",김영철 기자,2022-10-06,set(),million dollar evaporate land star usain bolt ...,Millions of dollars evaporated in the land sta...,1,212996,"{""Usain Bolt' (ORG)"", 'JAN (ORG)'}"


In [ ]:
print(len(df_ner_comp_no_set))
print(len(df))

163005
192206


In [ ]:
s = df_ner_comp_no_set['ner_orgs'].astype(str).str.split(',').explode().str.strip('{} ')


In [ ]:
s_df = pd.DataFrame(s)

In [ ]:
s_df['id'] = s_df.index + 1

In [ ]:
# strip quotation mark
s_df['ner_orgs'] = s_df['ner_orgs'].str.strip('\'')
s_df['ner_orgs'] = s_df['ner_orgs'].str.strip('\"')

# strop text + parantheses 
s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r"\(.*\)","")

# # strip space in the last character
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\'s')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\’s')


# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\'')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip()

# s_df['ner_orgs'] = s_df['ner_orgs'].str.strip('\'')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.strip('\"')

# # #
# # s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'“', '')

# remove 'a '
s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^the state ', '')
s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^The state ', '')
s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^the ', '')
s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^The ', '')
s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^an ', '')
s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^a ', '')

s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip()

# # strip space in the last character
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\'s')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\’s')



# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\'s')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\’s')

# s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^\'', '')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^\‘', '')

# s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^\“', '')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^’s ', '')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^• ', '')


# s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^\'', '')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\'')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\'s')
# s_df['ner_orgs'] = s_df['ner_orgs'].str.rstrip('\’s')

#

<ipython-input-39-b7b21fc4396d>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r"\(.*\)","")
<ipython-input-39-b7b21fc4396d>:23: FutureWarning: The default value of regex will change from True to False in a future version.
  s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^the state ', '')
<ipython-input-39-b7b21fc4396d>:24: FutureWarning: The default value of regex will change from True to False in a future version.
  s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^The state ', '')
<ipython-input-39-b7b21fc4396d>:25: FutureWarning: The default value of regex will change from True to False in a future version.
  s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^the ', '')
<ipython-input-39-b7b21fc4396d>:26: FutureWarning: The default value of regex will change from True to False in a future version.
  s_df['ner_orgs'] = s_df['ner_orgs'].str.replace(r'^The ', '')
<ipython-input-39-b

In [ ]:
# drop duplicates
s_df_nodup = s_df.drop_duplicates()

In [ ]:
# minimal + delete the/a
print(len(s_df_nodup))
print(len(s_df))

2149643
2164469


In [ ]:
# minimal
print(len(s_df_nodup))
print(len(s_df))

1146941
1149027


In [ ]:
# delete the/a
print(len(s_df_nodup))
print(len(s_df))

1114636
1149027


In [ ]:
# not deleting the/a
print(len(s_df_nodup))
print(len(s_df))

1135858
1149027


In [ ]:
print(len(s_df_nodup))
print(len(s_df))

1114561
1149027


In [ ]:
df_ner_comp_no_set_st = df_ner_comp_no_set[['id', 'source', 'article_final']]

In [ ]:
df_ner_comp_no_set_st = pd.merge(df_ner_comp_no_set_st, s_df_nodup, how='left', on='id')

In [ ]:
df_ner_comp_no_set_st

,id,source,article_final,ner_orgs
0,1,285 South,The GOP effort to court Hispanic voters: Repub...,Latinos
1,1,285 South,The GOP effort to court Hispanic voters: Repub...,State Senate
2,1,285 South,The GOP effort to court Hispanic voters: Repub...,WABE Interview
3,1,285 South,The GOP effort to court Hispanic voters: Repub...,Georgia Latino Alliance for Human Rights
4,1,285 South,The GOP effort to court Hispanic voters: Repub...,Emory University
...,...,...,...,...
2149638,212994,WQXI AM 790,Atlanta Radio Korea Atlanta Radio Korea FM96.7...,Atlanta Radio Korea
2149639,212995,WQXI AM 790,Atlanta Radio Korea Atlanta Radio Korea FM96.7...,Atlanta Radio Korea
2149640,212996,WQXI AM 790,Millions of dollars evaporated in the land sta...,Usain Bolt'
2149641,212996,WQXI AM 790,Millions of dollars evaporated in the land sta...,JAN


In [ ]:
df_ner_comp_no_set_st

,id,source,article_final,ner_orgs
0,1,Metter Advertiser,The American Pickers are excited to return to ...,American Pickers
1,1,Metter Advertiser,The American Pickers are excited to return to ...,CDC
2,1,Metter Advertiser,The American Pickers are excited to return to ...,American Pickers TV Show
3,2,Metter Advertiser,Candler County Health Department and Southeast...,SE Broad Street and
4,2,Metter Advertiser,Candler County Health Department and Southeast...,Southeast Health Unit
...,...,...,...,...
1126717,192185,WALB TV 10 (NBC),Meagan Muhlbach stood by and let herself get h...,Children’s Miracle Network
1126718,192189,WALB TV 10 (NBC),Saturday marked the sixth time in the series h...,Crimson Tide
1126719,192193,WALB TV 10 (NBC),Lexington Police are investigating the city's ...,Lexington Police
1126720,192194,WALB TV 10 (NBC),"Members and volunteers of Junior Achievement, ...",Junior Achievement


In [ ]:
ner_art_count = df_ner_comp_no_set_st.groupby('ner_orgs')['id'].count().reset_index()

In [ ]:
ner_art_count_c5 = ner_art_count[ner_art_count['id'] >= 5]

In [ ]:
ner_art_count_c5 = ner_art_count_c5.sort_values('ner_orgs')

In [ ]:
ner_art_count_c5 = ner_art_count_c5.reset_index()

In [ ]:
ner_art_count_c5 = ner_art_count_c5.iloc[0:34641,]

In [ ]:
ner_art_count_c5

,ner_orgs,id
0,,284
24,& Movement Disorders Support Group,23
29,& SI’s,33
39,& ©,24
44,'American Idol',5
...,...,...
154800,• Standalone Prescription Drug Plans,17
154826,• Trust,17
154852,•Danielsville Evangelical Church,8
154861,•Lighthouse Pentecostal Church,6


In [ ]:
ner_art_count_c5.to_excel('Georgia_ner_orgs_art_c5_v1_pre.xlsx')

In [ ]:
ner_art_count_c5 = ner_art_count_c5.sort_values('ner_orgs', ascending=False)

In [ ]:
35647-1006

34641

In [ ]:
34514-1006

33508

In [ ]:
ner_art_count_c5

,index,ner_orgs,id
35646,185839,🟣,8
35645,185829,🎥,15
35644,185827,🌽 @daboot02 @coach_frost @HuskerFBNation,6
35643,185804,−6.5,5
35642,185803,−6.0,8
...,...,...,...
1010,5479,Air Force and,5
1009,5478,Air Force Veteran,7
1008,5473,Air Force Security Forces,10
1007,5470,Air Force Reserve Command,7


In [ ]:
ner_art_count_c5.to_excel('Georgia_ner_orgs_art_c5_v2_add.xlsx')

## org topic count

In [ ]:
%cd /content/drive/MyDrive/Georgia/Organization Analysis


In [ ]:
import pandas as pd
org_v2 = pd.read_excel('Georgia_ner_orgs_art_c5_v2.xlsx')
org_v1 = pd.read_excel('Georgia_ner_orgs_art_c5_v1_pre.xlsx')

/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Organization Analysis


In [ ]:
org_v1

,Unnamed: 0,ner_orgs,id
0,0,NaN,284
1,24,& Movement Disorders Support Group,23
2,29,& SI’s,33
3,39,& ©,24
4,44,'American Idol',5
...,...,...,...
21371,154800,• Standalone Prescription Drug Plans,17
21372,154826,• Trust,17
21373,154852,•Danielsville Evangelical Church,8
21374,154861,•Lighthouse Pentecostal Church,6


In [ ]:
org_v1['version'] = 'v1'
org_v2['version'] = 'v2'

In [ ]:
org_merge = pd.merge(org_v1, org_v2, how='outer', on=["ner_orgs"])

In [ ]:
org_merge

,Unnamed: 0,ner_orgs,id_x,version_x,id_y,version_y
0,0.0,NaN,284.0,v1,27.0,v2
1,92026.0,NaN,28.0,v1,27.0,v2
2,24.0,& Movement Disorders Support Group,23.0,v1,NaN,NaN
3,29.0,& SI’s,33.0,v1,NaN,NaN
4,39.0,& ©,24.0,v1,NaN,NaN
...,...,...,...,...,...,...
35409,NaN,Zuckerman Museum of Art,NaN,NaN,5.0,v2
35410,NaN,Zwangobani,NaN,NaN,8.0,v2
35411,NaN,Zyan Royal,NaN,NaN,6.0,v2
35412,NaN,Zyon McCollum,NaN,NaN,10.0,v2


In [ ]:
org_merge_st = org_merge[['ner_orgs', 'version_x', 'version_y', 'id_y']]

In [ ]:
org_merge_repeat = org_merge_st[ (org_merge_st['version_x']=='v1') & (org_merge_st['version_y']=="v2")]

In [ ]:
org_merge_repeat

,ner_orgs,version_x,version_y,id_y
0,NaN,v1,v2,27.0
1,NaN,v1,v2,27.0
19,1.A Clear Understanding of Your Purpose,v1,v2,16.0
20,10 Downing Street,v1,v2,5.0
21,100k,v1,v2,16.0
...,...,...,...,...
20562,Zoological Society of London,v1,v2,6.0
20563,Zoom,v1,v2,12.0
20564,Zuckerberg,v1,v2,17.0
20565,Zulu,v1,v2,20.0


In [ ]:
org_merge_repeat.to_excel('Georgia_ner_orgs_art_c5_repeat.xlsx')

In [ ]:
import pandas as pd
%cd /content/drive/MyDrive/Georgia/Organization Analysis

df = pd.read_pickle('/content/drive/MyDrive/Georgia/Web Scraping/Georgia_data_v2_translation_ner2_lem.pkl')

org_merge_repeat = pd.read_excel('Georgia_ner_orgs_art_c5_repeat.xlsx')


/content/drive/.shortcut-targets-by-id/1YSQ9BBnwwRu7fyeM_9V6eoFgKQBPl4Rr/Georgia/Organization Analysis


In [ ]:
org_merge_repeat

,Unnamed: 0,ner_orgs,version_x,version_y,id_y
0,2902,CBS News,v1,v2,14337
1,290,AP,v1,v2,7110
2,2898,CBS NEWS,v1,v2,6480
3,12652,NFL,v1,v2,4877
4,1270,Associated Press,v1,v2,4286
...,...,...,...,...,...
19588,20532,Zatko $,v1,v2,5
19589,20542,Zichal,v1,v2,5
19590,20546,Zimmerman’s,v1,v2,5
19591,20549,Zion Custis,v1,v2,5


In [ ]:
df

,index,source,en_source,website,article_url,article_title,article_author,date_published,article_final,ner_places,ner_orgs,article_lemma_num,id
0,0,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_167b17...,American Pickers to film in Georgia,NaN,2022-09-19,The American Pickers are excited to return to ...,"{'America (GPE)', 'Georgia (GPE)'}","{'American Pickers (ORG)', 'CDC (ORG)', 'The A...",american picker excited return georgia plan fi...,1
1,1,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_248646...,Free Covid test kits available TODAY,NaN,2022-09-15,Candler County Health Department and Southeast...,{'Metter (GPE)'},"{'SE Broad Street and (ORG)', 'Southeast Healt...",candler county health department southeast hea...,2
2,2,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_2eccfd...,CLASS OF 2000 STILL LEADS THE WAY! Over $29K r...,NaN,2022-10-14,After the third week of the Education Foundati...,set(),"{'CarterFranklin (ORG)', 'Ronnie & Janie Sikes...",week education foundation 2022 campaign class ...,3
3,3,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_2f555e...,"MMS Lady Tigers to play for championship, MMS ...",NaN,2022-09-27,Congratulations to the Metter Middle School La...,"{'South Tattnall (GPE)', 'Bryan County (GPE)',...","{'MHS (ORG)', 'the Metter Middle School Lady T...",congratulation metter middle school lady tiger...,4
4,4,Metter Advertiser,1.0,http://www.metteradvertiser.com/,http://www.metteradvertiser.com/article_2fe7f1...,ATTN Hunters & Anglers: Celebrate Hunting & Fi...,NaN,2022-09-22,In a proclamation recently signed by Gov. Bria...,{'Georgia (GPE)'},"{'Outdoor Adventure Days (ORG)', 'Congressiona...",proclamation recently sign gov brian kemp satu...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192201,3074,WALB TV 10 (NBC),NaN,http://www.walb.com/,https://www.walb.com/video/2022/10/22/your-fir...,Your First Alert Forecast from late Sat. morni...,NaN,2022-10-14,Your First Alert Forecast from late Sat. morni...,set(),set(),alert forecast late sat morning oct 22 2022,192202
192202,3075,WALB TV 10 (NBC),NaN,http://www.walb.com/,https://www.walb.com/video/2022/10/22/your-fir...,"Your First Alert Forecast from Sat. morning, O...",NaN,2022-10-14,"Your First Alert Forecast from Sat. morning, O...",set(),set(),alert forecast sat morning oct 22 2022,192203
192203,3076,WALB TV 10 (NBC),NaN,http://www.walb.com/,https://www.walb.com/video/2022/10/24/organiza...,Organizations help provide financial relief fo...,NaN,2022-10-14,Some Nashville organizations are working to ma...,{'Nashville (GPE)'},set(),nashville organization work get financial help...,192204
192204,3077,WALB TV 10 (NBC),NaN,http://www.walb.com/,https://www.walb.com/video/2022/10/25/fentanyl...,Fentanyl fears prompts community roundtable in...,NaN,2022-09-28,"Parents, students, and professionals all gathe...",{'Glendale (GPE)'},set(),parent student professional gather school glen...,192205
